## Demo: Stage3 - Combine individual instruments to one xarray dataset

Here we want to treat one mooring as one dataset.

The time sampling differs between instruments -- for the *first guess* (24 Aug 2025) we will use a simple interpolation onto a common grid.



In [ ]:
import os

import yaml
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from datetime import datetime
from ctd_tools.writers import NetCdfWriter

from oceanarray import tools

moorlist = ['ds2_X_2012','ds2_X_2017','ds2_X_2018',
            'ds8_1_2012','ds9_1_2012','ds10_1_2012', 'ds11_1_2012','ds12_1_2012',
            'ds13_1_2012','ds14_1_2012','ds15_1_2012','ds16_1_2012','ds17_1_2012',
            'ds19_1_2012','ds18_1_2012','ds28_1_2017',
            'dsA_1_2018','dsB_1_2018','dsC_1_2018', 'dsD_1_2018','dsE_1_2018','dsF_1_2018',
            'dsM1_1_2017','dsM2_1_2017','dsM3_1_2017','dsM4_1_2017','dsM5_1_2017']
moorlist = ['dsE_1_2018']

## Load data for one mooring into datasets, list of xarray datasets

In [ ]:
from oceanarray.stage3 import Stage3Processor, process_multiple_moorings_stage3

# Simple usage
moorlist = ['dsE_1_2018']
basedir = '/Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/'
results = process_multiple_moorings_stage3(moorlist, basedir, file_suffix='_use')

In [ ]:
# Access individual results
print(results['dsE_1_2018'])  # True or False

# Check if a specific mooring succeeded
if results['dsE_1_2018']:
    print("dsE_1_2018 processed successfully")
else:
    print("dsE_1_2018 failed to process")

# Iterate through all results
for mooring_name, success in results.items():
    status = "SUCCESS" if success else "FAILED"
    print(f"{mooring_name}: {status}")

In [ ]:
# For a successful mooring, the combined data is saved as:
# {proc_dir}/{mooring_name}/{mooring_name}_mooring_use.nc

# Load the combined dataset
import xarray as xr

mooring_name = 'dsE_1_2018'
if results[mooring_name]:  # Only load if processing succeeded
    combined_file = f"{basedir}/moor/proc/{mooring_name}/{mooring_name}_mooring_use.nc"
    combined_ds = xr.open_dataset(combined_file)

    print(f"Combined dataset shape: {dict(combined_ds.dims)}")
    print(f"Variables: {list(combined_ds.data_vars)}")
    print(f"Instruments: {combined_ds.nominal_depth.values}")

In [ ]:
# The combined dataset has structure like:
# Dimensions: (time: 8640, N_LEVELS: 3)
# Variables: temperature(time, N_LEVELS), salinity(time, N_LEVELS), etc.

# Access temperature data for all instruments
temp_data = combined_ds.temperature  # Shape: (time, N_LEVELS)

# Get temperature for a specific instrument level
temp_level_0 = combined_ds.temperature[:, 0]  # First instrument
temp_level_1 = combined_ds.temperature[:, 1]  # Second instrument

# Access metadata for each level
depths = combined_ds.nominal_depth.values
serials = combined_ds.serial_number.values
clock_offsets = combined_ds.clock_offset.values

print(f"Instrument depths: {depths}")
print(f"Serial numbers: {serials}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_mooring_temperature(combined_ds, mooring_name):
    """Plot temperature records from all instruments on a mooring."""

    # Extract metadata for legend labels
    instruments = combined_ds.instrument_id.values if 'instrument_id' in combined_ds else None
    serials = combined_ds.serial_number.values
    depths = combined_ds.nominal_depth.values

    # Get instrument names from global attributes if available
    if 'instrument_names' in combined_ds.attrs:
        instrument_names = combined_ds.attrs['instrument_names'].split(', ')
    else:
        instrument_names = ['unknown'] * len(serials)

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 6))

    # Plot each instrument's temperature record
    for i in range(combined_ds.dims['N_LEVELS']):
        temp_data = combined_ds.temperature[:, i]

        # Skip if all NaN
        if np.all(np.isnan(temp_data)):
            continue

        # Create legend label
        if i < len(instrument_names):
            instrument_type = instrument_names[i]
        else:
            instrument_type = 'unknown'

        label = f"{instrument_type}:{int(serials[i])} ({depths[i]:.0f}m)"

        # Plot the data
        ax.plot(combined_ds.time, temp_data, label=label, linewidth=0.8)

    # Formatting
    ax.set_xlabel('Time')
    ax.set_ylabel('Temperature (°C)')
    ax.set_title(f'{mooring_name} - Temperature Records')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(True, alpha=0.3)

    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45)
    plt.tight_layout()

    return fig, ax

# Usage example:
mooring_name = 'dsE_1_2018'
if results[mooring_name]:
    combined_file = f"{basedir}/moor/proc/{mooring_name}/{mooring_name}_mooring_use.nc"
    combined_ds = xr.open_dataset(combined_file)

    fig, ax = plot_mooring_temperature(combined_ds, mooring_name)
    plt.show()

    # Optional: save the plot
    # plt.savefig(f'{mooring_name}_temperature_timeseries.png', dpi=150, bbox_inches='tight')